In [22]:
from ase.io import vasp
import numpy as np 
import pandas as pd

In [25]:
! grep "POTCAR:    PAW_PBE" OUTCAR

 POTCAR:    PAW_PBE Ti 08Apr2002                  
 POTCAR:    PAW_PBE C 08Apr2002                   
 POTCAR:    PAW_PBE O 08Apr2002                   
 POTCAR:    PAW_PBE W 08Apr2002                   
 POTCAR:    PAW_PBE S 06Sep2000                   
 POTCAR:    PAW_PBE Ti 08Apr2002                  
 POTCAR:    PAW_PBE C 08Apr2002                   
 POTCAR:    PAW_PBE O 08Apr2002                   
 POTCAR:    PAW_PBE W 08Apr2002                   
 POTCAR:    PAW_PBE S 06Sep2000                   


In [36]:
! grep "; ZVAL   =" OUTCAR
# ! grep "; ZVAL   =" OUTCAR | awk '{print $6}'

   POMASS =   47.880; ZVAL   =    4.000    mass and valenz                      
   POMASS =   12.011; ZVAL   =    4.000    mass and valenz                      
   POMASS =   16.000; ZVAL   =    6.000    mass and valenz                      
   POMASS =  183.850; ZVAL   =    6.000    mass and valenz                      
   POMASS =   32.066; ZVAL   =    6.000    mass and valenz                      


In [39]:
ZVALs={
    "Ti":4,
    "C":4,
    "O":6,
    "W":6,
    "S":6
}
atoms = vasp.read_vasp("./POSCAR")
data = pd.read_csv("./ACF.dat",skiprows=[1],delimiter=" ",skipinitialspace=True,nrows=len(atoms))

In [51]:
symbols = atoms.get_chemical_symbols()
print("Positive value : Charge Gain")
print("Negative value : Charge Loss\n")
for i,symbol in enumerate(symbols):
    zval=ZVALs[symbol]
    baderCharge=data["CHARGE"][i]
    print(f"Charge on {symbol:<2} is {baderCharge-zval:>6.3f}")
# np.sum(data["CHARGE"])

Positive value : Charge Gain
Negative value : Charge Loss

Charge on Ti is -1.507
Charge on Ti is -1.716
Charge on Ti is -1.715
Charge on C  is  1.490
Charge on C  is  1.493
Charge on O  is  0.996
Charge on O  is  0.981
Charge on W  is -1.165
Charge on S  is  0.567
Charge on S  is  0.575
